## 0. 필요 라이브러리 import

In [ ]:
%pip install google-cloud-storage

In [3]:
from google.oauth2 import service_account
from google.cloud import storage
import pandas as pd

## 1. GCP 권한 인증

In [2]:
## 권한 인증

JSON_KEY_PATH = "sprintda05-soomin-389e878cb2dc.json"

credentials = service_account.Credentials.from_service_account_file(JSON_KEY_PATH)

In [5]:
## Client 생성
project_id = 'sprintda05-soomin'
location = 'asia-northeast3'

client = storage.Client(
    project=project_id,
    credentials=credentials   
)

## 2. Parquet 파일 다루기
- 데이터를 객체 저장소에 저장할 때는, 용량 효율성, 빅데이터 도구들과의 호환성, 파티셔닝 등의 이점 때문에 CSV나 JSON 같은 포맷보다 Parquet 포맷을 사용하는 경우가 많습니다. (대부분 Parquet 사용)

In [ ]:
## parquet 파일 읽기
df = pd.read_parquet('./dataset/member.parquet')
df.head()

,idx,sex,status,grade
0,100,남,유료회원,초1학년
1,1000,여,유료회원,초5학년
2,10000,여,유료회원,초6학년
3,100007,남,유료회원,초4학년
4,10001,남,유료회원,초2학년


In [16]:
## parquet 파일 쓰기
df.to_parquet(
    path='dataset/member_parquet.parquet',
    index=False,
    engine='pyarrow',
    compression='gzip'
)

## 3. Bucket 관련 작업

In [ ]:
## 1) 버킷 목록 가져오기

In [ ]:
## 2) 버킷 생성

In [ ]:
## 3) 버킷 정보 조회

In [ ]:
## 4) 버킷 삭제

## 4. 객체 관련 작업

In [ ]:
## 1) 객체 업로드 - 파일

In [ ]:
## 2) 객체 업로드 - DataFrame

In [ ]:
## 3) 객체 다운로드

In [ ]:
## 4) 폴더 내 객체 목록 가져오기

In [ ]:
## 5) 객체 유무 조회

In [ ]:
## 6) 객체 삭제

## 5. Pandas로 GCP 데이터 읽기
- GCS를 작업자의 Local Directory처럼 활용하여 데이터를 조회할 수 있는 라이브러리

1. 로컬 디렉토리에 저장된 파일 (read_csv, read_parquet)
2. 데이터베이스에 저장된 테이블 (read_sql)
3. 빅쿼리에 저장된 테이블 (read_gbq)
4. 클라우드 객체 저장소(GCS)에 저장된 파일 (read_csv, read_parquet)

In [13]:
pd.read_csv(
    filepath_or_buffer="gs://sprintda05-soomin-bucket/education/point_his.csv",
    storage_options={"token":JSON_KEY_PATH}
).head()

,idx,proc_ym,proc_ymd,point
0,87376,202304,20230401,1000
1,87599,202304,20230401,1000
2,87682,202304,20230401,1000
3,87555,202304,20230401,1000
4,87569,202304,20230401,1000


In [15]:
pd.read_parquet(
    path="gs://sprintda05-soomin-bucket/BigQuery/regdate.parquet",
    storage_options={"token":JSON_KEY_PATH}
).head()

,idx,regdate
0,1,20221206
1,2,20221206
2,3,20221206
3,4,20221206
4,5,20221206


In [ ]:
%pip install fsspec gcsfs

In [ ]:
df = pd.read_parquet('dataset/member.parquet')
df.head()

## 6. (빅)데이터 저장시 Parquet 파일을 사용하는 이유

- 용량! -> csv에 비해서 훨씬 작은 용량을 차지
- 비용! -> 같은 데이터 저장 시, 더 작은 용량을 차지하기 때문에 비용이 덜 발생
- metadata 같이 저장 -> 저장/읽어올 때 더 빠르고 편리
- 빅데이터 관련 도구들과 호환 -> kafka, spark, flink, trino, hive, iceberg 등 빅데이터 도구들고 높은 호환성
- 파티셔닝 기능 -> 객체 저장소에 저장될 때 자동으로 파티셔닝이 가능!    *파티셔닝 : 정보를 바탕으로 구분지어놓은 것 (ex:YMD)

In [ ]:
### 데이터가 굉장히 커지면?
'''
- DB나 빅쿼리에 테이블로 저장
    - 높은 비용이 발생!

- 객체 스토리지에 파일로 저장 (parquet)
    - 비교적 적은 비용
    - 다른 도구들과 호환성이 뛰어남
    - 이 데이터를 Transform 하여 테이블로 저장하는 것이 효율적

'''

In [ ]:
### 데이터 저장 시 바람직하지 않은 방법
'''
data/
    20250501.parquet
    20250502.parquet
    20250503.parquet
    20250504.parquet

'''

In [ ]:
### 데이터 저장 시 바람직한 방법 (계층형)
'''
data/
    - yyyy=2023/
            - mm=04/
                    - dd=01
                        - data.parquet
                    - dd=02
                        - data.parquet
                    - dd=03
                        - data.parquet

'''

In [ ]:
pd.read_csv(
    filepath_or_buffer="gs://sprintda05-soomin-bucket/gcp/gcs_part_csv",    #폴더 경로를 입력한 경우
    storage_options={"token":JSON_KEY_PATH}
)
# -> Error 발생. csv는 파일 단위로 읽어와야함

In [ ]:
pd.read_parquet(
    path="gs://sprintda05-soomin-bucket/gcp/gcs_part_parquet",    #폴더 경로를 입력한 경우
    storage_options={"token":JSON_KEY_PATH}
)
# -> 경로에 존재하는 모든 parquet을 탐색하여 read & merge 한 데이터로 반환
# 폴더 구조(yyyy/mm/dd)가 컬럼으로 들어감

,idx,proc_ym,proc_ymd,pointnm,yyyy,mm,dd
0,75178,202304,2023-04-01,한글 스피치,2023,4,1
1,7287,202304,2023-04-01,중학 1학년,2023,4,1
2,5547,202304,2023-04-01,중학 1학년,2023,4,1
3,80092,202304,2023-04-01,중학 1학년,2023,4,1
4,67940,202304,2023-04-01,중학 2학년,2023,4,1
...,...,...,...,...,...,...,...
44417,12257,202304,2023-04-30,밀크T 지오그래픽,2023,4,30
44418,12257,202304,2023-04-30,밀크T 지오그래픽,2023,4,30
44419,12257,202304,2023-04-30,밀크T 지오그래픽,2023,4,30
44420,66583,202304,2023-04-30,밀크T 지오그래픽,2023,4,30


In [ ]:
'''
Csv와 Parquet의 차이점
'''